In [1]:
from selenium import webdriver
import numpy as np
import pandas as pd
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from math import ceil
import os

In [2]:
def calculate_pi_max(items_per_page, remain_count):
    """
    Tính số page tối đa cần load.

    items_per_page: số sản phẩm hiển thị trên page đầu tiên
    remain_count: số sản phẩm còn lại chưa hiển thị (xem từ 'Xem thêm')
    """
    total_items = items_per_page + remain_count
    pi_max = ceil(total_items / items_per_page)
    pi_max -= 1
    return pi_max, total_items

In [3]:
url = "https://www.thegioididong.com/laptop" 
request_url = "https://www.thegioididong.com/laptop#c=44&o=13&pi={}" #&pi=0

service = Service(r"C:\web driver\msedgedriver.exe")
driver = webdriver.Edge(service=service)
driver.get(url)
sleep(3) 

items = driver.find_elements(By.CSS_SELECTOR, "a.main-contain")
remain_count = int(driver.find_elements(By.CSS_SELECTOR, "strong.see-more-btn span.remain")[0].text)

print("product limit per page:", len(items))
print("total products:", remain_count)
driver.quit()

product limit per page: 20
total products: 386


In [4]:
print(calculate_pi_max(len(items),remain_count))
pi = calculate_pi_max(len(items),remain_count)[0]

(20, 406)


In [5]:
url = request_url.format(pi)

driver = webdriver.Edge(service=service)
driver.get(url)
sleep(random.randint(5, 10))  # delay 

pid, links, names, prices, brands = [], [], [], [], []

items = driver.find_elements(By.CSS_SELECTOR, "a.main-contain")
for item in items:
    pid.append(item.get_attribute("data-id"))
    links.append(item.get_attribute("href"))
    names.append(item.get_attribute("data-name"))
    prices.append(item.get_attribute("data-price"))
    brands.append(item.get_attribute("data-brand"))
print(f"all page has been crawled, total product: {len(links)}")

driver.quit()

all page has been crawled, total product: 405


In [6]:
df = pd.DataFrame({
    "pid": pid,
    "link": links,
    "name": names,
    "price": prices,
    "brand": brands
})
df.head()

,pid,link,name,price,brand
0,327098,https://www.thegioididong.com/laptop/hp-15-fc0...,"Laptop HP 15 fc0085AU - A6VV8PA (R5 7430U, 16G...",12990000.0,HP
1,311178,https://www.thegioididong.com/laptop/asus-vivo...,Laptop Asus Vivobook Go 15 E1504FA - NJ776W (R...,12390000.0,Asus
2,334442,https://www.thegioididong.com/laptop/lenovo-id...,Laptop Lenovo IdeaPad Slim 3 15IRH10 - 83K1000...,15890000.0,Lenovo
3,335362,https://www.thegioididong.com/laptop/macbook-a...,Laptop MacBook Air 13 inch M4 16GB/256GB,25090000.0,MacBook
4,337470,https://www.thegioididong.com/laptop/dell-insp...,Laptop Dell Inspiron 15 3520 - 71064798 (i5 12...,15490000.0,Dell


In [7]:
os.makedirs("tgdd", exist_ok=True)
df.to_csv("tgdd/raw_data.csv", index=False)